In [60]:
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

In [61]:
# news api for collecting data
api_key = os.getenv("news_api")
newsapi = NewsApiClient(api_key=api_key)

In [62]:
# import our initial batch of 20 companies (10 bankrupt, 10 healthy)
filepath = Path("../Project_2/Stocks_MasterList_Test.csv")
stocks_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
stocks_df.head(41)

,Symbol,Company Name,Price Performance (4 Weeks),Sector,Market Capitalization,Binary
0,RHHBF,Roche Holding AG,-5.28026,Health Care,$2.39T,1
1,AAPL,Apple Inc,-8.40659,Information Technology,$2.09T,1
2,AMZN,Amazon.com Inc,-5.98070,Consumer Discretionary,$1.55T,1
3,BRK/B,Berkshire Hathaway Inc,5.01114,Financials,$589.66B,0
4,V,Visa Inc,6.33930,Information Technology,$477.13B,0
5,JPM,JPMorgan Chase & Co,9.99646,Financials,$473.79B,0


In [63]:
company_info_df = stocks_df[['Symbol', 'Company Name']].copy()
company_info_df.head(41)

,Symbol,Company Name
0,RHHBF,Roche Holding AG
1,AAPL,Apple Inc
2,AMZN,Amazon.com Inc
3,BRK/B,Berkshire Hathaway Inc
4,V,Visa Inc
5,JPM,JPMorgan Chase & Co


In [64]:
search_list = []

for index, row in company_info_df.iterrows():
    ticker = row['Symbol']
    company_name = row['Company Name']
    search_string = ticker + ' ' + "AND" + ' ' + company_name
    search_list.append(search_string)

print(search_list)

['RHHBF AND Roche Holding AG', 'AAPL AND Apple Inc', 'AMZN AND Amazon.com Inc', 'BRK/B AND Berkshire Hathaway Inc', 'V AND Visa Inc', 'JPM AND JPMorgan Chase & Co']


In [65]:
# this is not finished!!!

# we handled some of the data manipulation above, so the below funciton could be simplified

company_sentiments = []

for search_string in search_list:
    company_headlines = newsapi.get_everything(
        q = search_string, # here the function should iterate through each string in the list
        language = "en", # english
        page_size = 100, # not sure need to look at documentation
        sort_by = "relevancy") # outside of relevance I thnk we should look at limiting # of articles and date range
    
    # now that we gathered the articles for the first company in the list in "company_headlines" we need to conduct the vader sentiment
    for article in company_headlines["articles"]:
        try: 
            text = article["content"]
            date = article["publishedAt"][:10] # the has impact on timing of articles, need to look a documentation
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
        
            company_sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu})
        except AttributeError:
            pass
            
    # now that we ran the vader analysis, we need to grab the describe dataframe
    sentiment_df = pd.DataFrame(company_sentiments) # "company_sentiments" is the list from above
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    sentiment_df = sentiment_df[cols]
    sentiment_stats_df = sentiment_df.describe()
    
    ### update: I realized we do not need to append our sentiments directly to above dataframes, instead
    # it is probaby easier just to create a new dataframe here then concat later
    
    # now need to append the outcomes to a dataframe - this can be new dataframe we later conccat to our original DF
    final_sentiment_df = pd.DataFrame()
    final_sentiment_df["compound"] = sentiment_stats_df.at[1,'compound']
    final_sentiment_df["positive"] = sentiment_stats_df.at[1,'positive']
    final_sentiment_df["negative"] = sentiment_stats_df.at[1,'negative']
    final_sentiment_df["neutral"] = sentiment_stats_df.at[1,'neutral']
    
print(final_sentiment_df)

KeyError: "None of [Index(['date', 'text', 'compound', 'positive', 'negative', 'neutral'], dtype='object')] are in the [columns]"